In [1]:
import numpy as np
from model import *
from train import *
%load_ext autoreload
%autoreload 2
from data_generator import *

Using TensorFlow backend.


In [2]:
images_dir1 = 'datasets/CelebA'
images_dir2 = 'datasets/dogs'
batch_size = 2

In [3]:
E_G = Encoder_Generator(num_channels=3, resolution=32, **config.G)

inputs :  [(None, 16, 16, 256), (None, 16, 16, 256)]
inputs :  [(None, 8, 8, 512), (None, 8, 8, 512)]
inputs :  [(None, 4, 4, 512), (None, 4, 4, 512)]
Instructions for updating:
keep_dims is deprecated, use keepdims instead
inputs :  [(None, 32, 32, 3), (None, 16, 16, 3), (None, 8, 8, 3), (None, 4, 4, 3)]


In [4]:
E_G.compile(optimizers.Adam(lr=0.0),
          loss=cycle_consistency_loss)

data_generator1 = DataGenerator(images_dir=images_dir1)
data_generator2 = DataGenerator(images_dir=images_dir2)

generator1 = data_generator1.generate(batch_size=4, img_size=32)
generator2 = data_generator2.generate(batch_size=4, img_size=32)

real_1 = next(generator1)
real_2 = next(generator2)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
K.set_value(E_G.cur_lod,np.float32(0))

In [6]:
E_G.cur_lod

<tf.Variable 'cur_lod:0' shape=() dtype=float32_ref>

In [7]:
E_G.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Eimages (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
E4xNIN (Conv2D)                  (None, 32, 32, 128)   384         Eimages[0][0]                    
____________________________________________________________________________________________________
E4xNINWS (WScaleLayer)           (None, 32, 32, 128)   1.0         E4xNIN[0][0]                     
____________________________________________________________________________________________________
add_bias_layer_1 (AddBiasLayer)  (None, 32, 32, 128)   128         E4xNINWS[0][0]                   
___________________________________________________________________________________________

                                                                   activation_7[0][0]               
____________________________________________________________________________________________________
E2b (Conv2D)                     (None, 8, 8, 512)     2359296     E2lod[0][0]                      
____________________________________________________________________________________________________
E2bws (WScaleLayer)              (None, 8, 8, 512)     1.0         E2b[0][0]                        
____________________________________________________________________________________________________
add_bias_layer_8 (AddBiasLayer)  (None, 8, 8, 512)     512         E2bws[0][0]                      
____________________________________________________________________________________________________
activation_8 (Activation)        (None, 8, 8, 512)     0           add_bias_layer_8[0][0]           
___________________________________________________________________________________________

____________________________________________________________________________________________________
G2aBN (BatchNormalization)       (None, 8, 8, 512)     2048        activation_14[0][0]              
____________________________________________________________________________________________________
G2aPN (PixelNormLayer)           (None, 8, 8, 512)     0           G2aBN[0][0]                      
____________________________________________________________________________________________________
G2b (Conv2D)                     (None, 8, 8, 512)     2359296     G2aPN[0][0]                      
____________________________________________________________________________________________________
G2bWS (WScaleLayer)              (None, 8, 8, 512)     1.0         G2b[0][0]                        
____________________________________________________________________________________________________
add_bias_layer_15 (AddBiasLayer) (None, 8, 8, 512)     512         G2bWS[0][0]             

Glod3NIN (Conv2D)                (None, 4, 4, 3)       1536        G1aPN[0][0]                      
____________________________________________________________________________________________________
Glod0NINWS (WScaleLayer)         (None, 32, 32, 3)     1.0         Glod0NIN[0][0]                   
____________________________________________________________________________________________________
Glod1NINWS (WScaleLayer)         (None, 16, 16, 3)     1.0         Glod1NIN[0][0]                   
____________________________________________________________________________________________________
Glod2NINWS (WScaleLayer)         (None, 8, 8, 3)       1.0         Glod2NIN[0][0]                   
____________________________________________________________________________________________________
Glod3NINWS (WScaleLayer)         (None, 4, 4, 3)       1.0         Glod3NIN[0][0]                   
___________________________________________________________________________________________

In [8]:
E_G.predict(real_1)

InvalidArgumentError: Incompatible shapes: [8,32,32,3] vs. [4,32,32,3]
	 [[Node: Glod/add = Add[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Glod/mul, Glod/mul_1)]]
	 [[Node: Glod/cond_3/Merge/_261 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1758_Glod/cond_3/Merge", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Glod/add', defined at:
  File "C:\Users\Paul\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Paul\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Paul\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Paul\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_inst

In [2]:
E_G = model.Encoder_Generator( use_wscale=False)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [3]:
Et_Gt = model.new_batch_norm(E_G)

In [4]:
Et = model.extract_encoder(Et_Gt)
E = model.extract_encoder(E_G)


In [5]:
Et_G = model.replace_batch_norm(E_G, Et_Gt, apply = 'encoder')

E_Gt = model.replace_batch_norm(E_G, Et_Gt, apply = 'generator')

In [16]:
x = Sequential([E_Gt, E])
x.summary(), E_Gt.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_6 (Model)              (None, 32, 32, 1)         9328388   
_________________________________________________________________
model_4 (Model)              (None, 4, 4, 256)         5189632   
Total params: 9,328,388
Trainable params: 9,320,708
Non-trainable params: 7,680
_________________________________________________________________
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Eimages (InputLayer)             (None, 32, 32, 1)     0                                            
____________________________________________________________________________________________________
E4xNIN (Conv2D)                  (None, 32, 32, 256)   512         Eimages[0][0]                    
___________________

(None, None)

In [12]:
from keras.models import Model, Sequential
E_G_twin_E = Model(inputs = E_Gt.input, outputs= E(E_Gt).output)

ValueError: Layer model_4 was called with an input that isn't a symbolic tensor. Received type: <class 'keras.engine.training.Model'>. Full input: [<keras.engine.training.Model object at 0x000002339EE94F28>]. All inputs to the layer should be tensors.

In [10]:
E_G.get_layer('G1a')(Et.output)

<tf.Tensor 'G1a_6/sub:0' shape=(?, 4, 4, 256) dtype=float32>

In [3]:
from keras.models import Model, Sequential

G = Model(inputs=E_G.get_layer('G1a').input, outputs=E_G.output)

C:\Users\Paul\Anaconda3\lib\site-packages\keras\engine\topology.py:1571: UserWarning: Model inputs must come from a Keras Input layer, they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to "model_2" was not an Input tensor, it was generated by layer E1aBN.
Note that input tensors are instantiated via `tensor = Input(shape)`.
The tensor that caused the issue was: E1aBN/cond/Merge:0
  str(x.name))


TypeError: Input layers to a `Model` must be `InputLayer` objects. Received inputs: Tensor("E1aBN/cond/Merge:0", shape=(?, 4, 4, 256), dtype=float32). Input 0 (0-based) originates from layer type `BatchNormalization`.

In [13]:
Et_G.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Eimages (InputLayer)             (None, 32, 32, 1)     0                                            
____________________________________________________________________________________________________
E4xNIN (Conv2D)                  (None, 32, 32, 256)   512         Eimages[0][0]                    
____________________________________________________________________________________________________
E4b (Conv2D)                     (None, 32, 32, 256)   590080      E4xNIN[1][0]                     
____________________________________________________________________________________________________
E4bBN_twin (BatchNormalization)  (None, 32, 32, 256)   1024        E4b[1][0]                        
___________________________________________________________________________________________

In [14]:
from keras.models import Sequential
Sequential([G, D])

In [10]:
D = model.Discriminator(resolution=32)

In [16]:
D.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Dimages (InputLayer)             (None, 16, 16, 1)     0                                            
____________________________________________________________________________________________________
D3xNIN (Conv2D)                  (None, 16, 16, 256)   256         Dimages[0][0]                    
____________________________________________________________________________________________________
D3xNINWS (WScaleLayer)           (None, 16, 16, 256)   1.0         D3xNIN[0][0]                     
____________________________________________________________________________________________________
add_bias_layer_73 (AddBiasLayer) (None, 16, 16, 256)   256         D3xNINWS[0][0]                   
___________________________________________________________________________________________

In [3]:
tw = model.replace_batch_norm(G)

In [21]:
enc = model.Encoder()

In [6]:
enc_tw = model.replace_batch_norm(enc)

In [7]:
enc.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Dimages (InputLayer)             (None, 32, 32, 1)     0                                            
____________________________________________________________________________________________________
D4xNIN (Conv2D)                  (None, 32, 32, 256)   256         Dimages[0][0]                    
____________________________________________________________________________________________________
D4xNINWS (WScaleLayer)           (None, 32, 32, 256)   1.0         D4xNIN[0][0]                     
____________________________________________________________________________________________________
add_bias_layer_1 (AddBiasLayer)  (None, 32, 32, 256)   256         D4xNINWS[0][0]                   
___________________________________________________________________________________________